# DASR

In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error

def concordance_correlation_coefficient(y_true, y_pred):
    """Concordance correlation coefficient."""
    # Raw data
    dct = {
        'y_true': y_true,
        'y_pred': y_pred
    }
    df_others = pd.DataFrame(dct)
    # Remove NaNs
    df_others = df_others.dropna()
    # Pearson product-moment correlation coefficients
    y_true = df_others['y_true']
    y_pred = df_others['y_pred']
    cor = np.corrcoef(y_true, y_pred)[0][1]
    # Means
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)
    # Population variances
    var_true = np.var(y_true)
    var_pred = np.var(y_pred)
    # Population standard deviations
    sd_true = np.std(y_true)
    sd_pred = np.std(y_pred)
    # Calculate CCC
    numerator = 2 * cor * sd_true * sd_pred
    denominator = var_true + var_pred + (mean_true - mean_pred)**2
    return numerator / denominator

def get_stats(string_name):
    print(mean_squared_error(df_others["mGFR"],df_others[string_name]), sum(np.equal(df_others["mGFR"]>60, df_others[string_name]>60)), concordance_correlation_coefficient(df_others["mGFR"],df_others[string_name]),
          sum((np.abs(df_others[string_name]-df_others["mGFR"])/df_others["mGFR"])<0.1),
          sum((np.abs(df_others[string_name]-df_others["mGFR"])/df_others["mGFR"])<0.3)
         )

In [2]:
from DASR import *
import numpy as np
from tqdm import tqdm
import pandas as pd
from collections.abc import Iterable

In [3]:
symbol_list = ["Pow","Add","Sub","Mul","Div","age","R","height","weight"]
orig = ("Add","Div","R","age","R")
orig2 = ("Add","Div","R","height","R")
orig3 = ("Add","Div","R","weight","R")
exp_set = []
for orig_eq in [orig,orig2,orig3]:
    for index1 in range(len(orig_eq)):
        for index2 in range(index1+1,len(orig_eq)):
            temp_list = list(orig_eq)
            for j in symbol_list:
                temp_list[index1] = j
                for k in symbol_list:
                    temp_list[index2] = k
                    exp_set.append(tuple(temp_list))    
exp_set = [tree_to_exp(kexp_to_tree(list(i)+["R"]*(max([len(i) for i in [orig,orig2,orig3]])+1))) for i in exp_set]
exp_set = sorted(set(exp_set))

In [4]:
exp_set

['Add(Add(Add(R,R),R),R)',
 'Add(Add(Add(R,R),age),R)',
 'Add(Add(Add(R,R),height),R)',
 'Add(Add(Add(R,R),weight),R)',
 'Add(Add(Div(R,R),R),R)',
 'Add(Add(Div(R,R),age),R)',
 'Add(Add(Div(R,R),height),R)',
 'Add(Add(Div(R,R),weight),R)',
 'Add(Add(Mul(R,R),R),R)',
 'Add(Add(Mul(R,R),age),R)',
 'Add(Add(Mul(R,R),height),R)',
 'Add(Add(Mul(R,R),weight),R)',
 'Add(Add(Pow(R,R),R),R)',
 'Add(Add(Pow(R,R),age),R)',
 'Add(Add(Pow(R,R),height),R)',
 'Add(Add(Pow(R,R),weight),R)',
 'Add(Add(R,R),Div(Add(R,R),R))',
 'Add(Add(R,R),Div(Div(R,R),R))',
 'Add(Add(R,R),Div(Mul(R,R),R))',
 'Add(Add(R,R),Div(Pow(R,R),R))',
 'Add(Add(R,R),Div(R,R))',
 'Add(Add(R,R),Div(Sub(R,R),R))',
 'Add(Add(R,R),Div(age,Add(R,R)))',
 'Add(Add(R,R),Div(age,Div(R,R)))',
 'Add(Add(R,R),Div(age,Mul(R,R)))',
 'Add(Add(R,R),Div(age,Pow(R,R)))',
 'Add(Add(R,R),Div(age,R))',
 'Add(Add(R,R),Div(age,Sub(R,R)))',
 'Add(Add(R,R),Div(age,age))',
 'Add(Add(R,R),Div(age,height))',
 'Add(Add(R,R),Div(age,weight))',
 'Add(Add(R,R),

In [5]:
len(exp_set)

873

In [9]:
df = pd.read_csv("df_2308.csv")
df = df[["age_cal","height","weight","airway_tube_size"]]
df.columns = ["age","height","weight","airway_tube_size"]
df = df.dropna()
df

,age,height,weight,airway_tube_size
0,8.542270,134.2,34.40,6.5
1,1.034929,78.0,10.10,4.0
3,6.612045,127.3,23.05,6.0
6,1.248486,76.4,11.00,4.0
8,12.777812,142.3,53.25,6.5
...,...,...,...,...
32043,0.372355,67.0,5.10,3.5
32045,6.434081,119.9,20.90,5.0
32046,8.972121,137.0,28.00,5.5
32047,5.084293,111.7,18.75,5.0


In [10]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from tqdm import tqdm
import itertools
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import numpy as np
from sklearn.model_selection import train_test_split
from pmlb import fetch_data, regression_dataset_names
import time


def cost(x, xdata, ydata, lambda_string): 
    y_pred = eval(lambda_string)(x, xdata)
    return np.mean(((y_pred - ydata))**2) 

def cost2(x, xdata, ydata, lambda_string): 
    y_pred = eval(lambda_string)(x, xdata)
    return concordance_correlation_coefficient(ydata,y_pred)

def cust_pred(x, xdata, ydata, lambda_string): 
    y_pred = eval(lambda_string)(x, xdata)
    return y_pred

def random_constant():
        return np.random.randint(1, 31)/10

In [11]:
X, y = np.array(df.iloc[:,:-1]).T, np.array(df.iloc[:,-1])
num_of_feature = X.shape[1]
mse_tuple = tuple()
np.random.seed(0)
master_list = []
for repeat_trial in range(1):
    for test_eq in tqdm(exp_set):
        test_eq_orig = test_eq
        R_count = test_eq.count("R")
        if R_count==0:
            continue
        lambda_string = "lambda x,xdata:"
        index = 0    
        for i in range(R_count):
            test_eq = test_eq.replace("R", f"x[{i}]", 1)
        lambda_string += test_eq
        optimized_cost = 0
        ypred =[]
        ytrue =[]
        lambda_string_list = []
        res_x_list =[]
        try:
            for xdata, ydata in zip([X],[y]):
                lambda_string_new = lambda_string.replace("height","(xdata[1])")
                lambda_string_new = lambda_string_new.replace("age","(xdata[0])")
                lambda_string_new = lambda_string_new.replace("weight","(xdata[2])")
                res = minimize(cost,
                               x0 = [random_constant() for i in range(R_count)],
                               args=(xdata,ydata,lambda_string_new),
                               method="BFGS")                      
                optimized_cost = cost(res.x, xdata, ydata, lambda_string_new)
            optimized_cost = optimized_cost*(1/len(y))
            master_list.append((test_eq_orig,test_eq,lambda_string_new,res.x,optimized_cost))
        except RuntimeError:
            print("No fit found")

100%|████████████████████████████████████████████████████████████████████████████████| 873/873 [13:06<00:00,  1.11it/s]


In [12]:
sorted(master_list,key=lambda x: x[-1])

[('Add(Pow(age,R),Pow(R,R))',
  'Add(Pow(age,x[0]),Pow(x[1],x[2]))',
  'lambda x,xdata:Add(Pow((xdata[0]),x[0]),Pow(x[1],x[2]))',
  array([0.48872132, 3.75099387, 0.85763306]),
  4.909772605612621e-06),
 ('Add(Div(R,R),Pow(age,R))',
  'Add(Div(x[0],x[1]),Pow(age,x[2]))',
  'lambda x,xdata:Add(Div(x[0],x[1]),Pow((xdata[0]),x[2]))',
  array([1.00007952, 0.32183084, 0.48872131]),
  4.909772605613428e-06),
 ('Add(Add(R,R),Pow(age,R))',
  'Add(Add(x[0],x[1]),Pow(age,x[2]))',
  'lambda x,xdata:Add(Add(x[0],x[1]),Pow((xdata[0]),x[2]))',
  array([0.9037346 , 2.2037346 , 0.48872151]),
  4.909772605623933e-06),
 ('Add(Add(R,R),Pow(height,R))',
  'Add(Add(x[0],x[1]),Pow(height,x[2]))',
  'lambda x,xdata:Add(Add(x[0],x[1]),Pow((xdata[1]),x[2]))',
  array([-1.60251705, -0.70251705,  0.43275881]),
  4.943636934869172e-06),
 ('Add(Div(R,R),Pow(height,R))',
  'Add(Div(x[0],x[1]),Pow(height,x[2]))',
  'lambda x,xdata:Add(Div(x[0],x[1]),Pow((xdata[1]),x[2]))',
  array([-54.50700418,  23.64695995,   0.43

# DASR-HE

In [13]:
X, y = np.array(df.iloc[:,:-1]).T, np.array(df.iloc[:,-1])
num_of_feature = X.shape[1]
mse_tuple = tuple()
np.random.seed(0)
master_list = []
for repeat_trial in range(1):
    for test_eq in tqdm(exp_set):
        test_eq_orig = test_eq
        R_count = test_eq.count("R")
        if R_count==0:
            continue
        lambda_string = "lambda x,xdata:"
        index = 0    
        for i in range(R_count):
            test_eq = test_eq.replace("R", f"x[{i}]", 1)
        lambda_string += test_eq
        optimized_cost = 0
        ypred =[]
        ytrue =[]
        lambda_string_list = []
        res_x_list =[]
        x0_base = [random_constant() for i in range(R_count)]
        try:
            for xdata, ydata in zip([X],[y]):
                lambda_string_new = lambda_string.replace("height","(xdata[1])")
                lambda_string_new = lambda_string_new.replace("age","(xdata[0])")
                lambda_string_new = lambda_string_new.replace("weight","(xdata[2])")
                x0 = x0_base.copy()
                cur_cost = cost(x0, xdata, ydata, lambda_string_new)
                for step_perc, rounds, absolute in zip([0.1, 0.1, 0.01],[10,10,10],[True, False, False]):
                    for _ in range(rounds):
                        for x0_idx in range(len(x0)):
                            test_x0 = x0.copy()
                            if absolute:
                                test_x0[x0_idx] = x0[x0_idx]+step_perc
                            else:
                                test_x0[x0_idx] = x0[x0_idx]*(1+step_perc)
                            new_cost = cost(test_x0, xdata, ydata, lambda_string_new)
                            if absolute:
                                outcome = step_perc*(new_cost<cur_cost)                                
                            else:
                                outcome = test_x0[x0_idx]*step_perc*(new_cost<cur_cost)
                            x0[x0_idx] = x0[x0_idx] + outcome
                            cur_cost = cost(x0, xdata, ydata, lambda_string_new)

                        for x0_idx in range(len(x0)):
                            test_x0 = x0.copy()
                            if absolute:
                                test_x0[x0_idx] = x0[x0_idx]-step_perc
                            else:
                                test_x0[x0_idx] = x0[x0_idx]*(1-step_perc)
                            new_cost = cost(test_x0, xdata, ydata, lambda_string_new)
                            if absolute:
                                outcome = step_perc*(new_cost<cur_cost)                                
                            else:
                                outcome = test_x0[x0_idx]*step_perc*(new_cost<cur_cost)
                            x0[x0_idx] = x0[x0_idx] - outcome
                            cur_cost = cost(x0, xdata, ydata, lambda_string_new)       
                optimized_cost = cost(x0, xdata, ydata, lambda_string_new)
            optimized_cost = optimized_cost*(1/len(y))
            master_list.append((test_eq_orig,test_eq,lambda_string_new,x0,optimized_cost))
        except RuntimeError:
            print("No fit found")

100%|████████████████████████████████████████████████████████████████████████████████| 873/873 [23:28<00:00,  1.61s/it]


In [14]:
sorted(master_list,key=lambda x: x[-1])

[('Add(Pow(age,R),Pow(R,R))',
  'Add(Pow(age,x[0]),Pow(x[1],x[2]))',
  'lambda x,xdata:Add(Pow((xdata[0]),x[0]),Pow(x[1],x[2]))',
  [0.49014900499999997, 2.8000000000000003, 1.1],
  4.910416603070819e-06),
 ('Add(Pow(age,Sub(R,R)),R)',
  'Add(Pow(age,Sub(x[0],x[1])),x[2])',
  'lambda x,xdata:Add(Pow((xdata[0]),Sub(x[0],x[1])),x[2])',
  [0.378585811481803, -0.11154152109679867, 3.1020809989499027],
  4.910453882367391e-06),
 ('Add(Pow(age,Div(R,R)),R)',
  'Add(Pow(age,Div(x[0],x[1])),x[2])',
  'lambda x,xdata:Add(Pow((xdata[0]),Div(x[0],x[1])),x[2])',
  [1.5543916906631206, 3.162936231000001, 3.096005634674638],
  4.912541692533803e-06),
 ('Add(Add(R,R),Pow(age,R))',
  'Add(Add(x[0],x[1]),Pow(age,x[2]))',
  'lambda x,xdata:Add(Add(x[0],x[1]),Pow((xdata[0]),x[2]))',
  [0.6419757911463772, 2.4534446738255995, 0.49194108599999986],
  4.9132975558270385e-06),
 ('Add(Pow(age,R),R)',
  'Add(Pow(age,x[0]),x[1])',
  'lambda x,xdata:Add(Pow((xdata[0]),x[0]),x[1])',
  [0.49019802480199, 3.0887293

In [15]:
import tenseal as ts

# controls precision of the fractional part
bits_scale = 40
multiplicative_depth = 7

# Create TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=16384,
    coeff_mod_bit_sizes=[bits_scale+20] + [bits_scale]*multiplicative_depth + [bits_scale+20]
)

# set the scale
context.generate_galois_keys()
context.global_scale = pow(2, bits_scale)

def cost(x, xdata, ydata, lambda_string): 
    y_pred = eval(lambda_string)(x, xdata)
    return (((y_pred - ydata))**2).sum()

def Pow(op1, op2):
    if type(op1)==ts.tensors.ckksvector.CKKSVector:
        op1 = op1.decrypt()
    if type(op2)==ts.tensors.ckksvector.CKKSVector:
        op2 = op2.decrypt()
    if (not isinstance(op1, Iterable)) and (not isinstance(op2, Iterable)):
        if op1>0:
            return ts.ckks_vector(context, [op1**op2])
        else:
            return ts.ckks_vector(context, [-((-op1)**op2)])
    if not isinstance(op1, Iterable):
        op1 = [op1]*len(op2)
    elif len(op1) == 1:
        op1 = [op1[0]]*len(op2)
    if not isinstance(op2, Iterable):
        op2 = [op2]*len(op1)
    elif len(op2) == 1:
        op2 = [op2[0]]*len(op1)
    temp_list = []
    for i, j in zip(op1,op2):
        if i>0:
            temp_list.append(i**j)
        else:
            temp_list.append(-((-i)**j))
    return ts.ckks_vector(context,temp_list)

def Mul(op1, op2):
    return op1 * op2

def Add(op1, op2):
    return op1 + op2

def Sub(op1, op2):
    return op1 - op2

def Div(op1, op2):
    if type(op1)==ts.tensors.ckksvector.CKKSVector:
        op1 = np.array(op1.decrypt())
    if type(op2)==ts.tensors.ckksvector.CKKSVector:
        op2 = np.array(op2.decrypt())
    if isinstance(op1, Iterable) or isinstance(op2, Iterable):
        return ts.ckks_vector(context, op1 / op2)
    else:
        return ts.ckks_vector(context, [op1 / op2])
    
def random_constant():
        return np.random.randint(1, 31)/10
    
X, y = np.array(df.iloc[:5000,:-1]).T, np.array(df.iloc[:5000,-1]).astype(float)
X1 = [ts.ckks_vector(context, x.tolist()) for x in X]
y1 = ts.ckks_vector(context, y)

num_of_feature = X.shape[1]
mse_tuple = tuple()
np.random.seed(0)
master_list = []
for repeat_trial in range(1):
    for test_eq in ['Add(Pow(age,R),Pow(R,R))']:
        test_eq_orig = test_eq
        R_count = test_eq.count("R")
        if R_count==0:
            continue
        lambda_string = "lambda x,xdata:"
        index = 0    
        for i in range(R_count):
            test_eq = test_eq.replace("R", f"x[{i}]", 1)
        lambda_string += test_eq
        optimized_cost = 0
        ypred =[]
        ytrue =[]
        lambda_string_list = []
        res_x_list =[]
        x0_base = [random_constant() for i in range(R_count)]
        x0_base = [ts.ckks_vector(context,[i]) for i in x0_base]
        try:
            for xdata, ydata in zip([X1],[y1]):
                lambda_string_new = lambda_string.replace("height","(xdata[1])")
                lambda_string_new = lambda_string_new.replace("age","(xdata[0])")
                lambda_string_new = lambda_string_new.replace("weight","(xdata[2])")
                x0 = x0_base.copy()
                cur_cost = cost(x0, xdata, ydata, lambda_string_new)
                for step_perc, rounds, absolute in zip([0.1, 0.1, 0.01],[10,10,10],[True, False, False]):
                    for _ in tqdm(range(rounds)):
                        for x0_idx in range(len(x0)):
                            test_x0 = x0.copy()
                            if absolute:
                                test_x0[x0_idx] = x0[x0_idx]+step_perc
                            else:
                                test_x0[x0_idx] = x0[x0_idx]*(1+step_perc)
                            new_cost = cost(test_x0, xdata, ydata, lambda_string_new)
                            if absolute:
                                outcome = ts.ckks_vector(context, [step_perc*(new_cost.decrypt()[0]<cur_cost.decrypt()[0])])                              
                            else:
                                outcome = ts.ckks_vector(context, [x0[x0_idx].decrypt()[0]*step_perc*(new_cost.decrypt()[0]<cur_cost.decrypt()[0])])
                            x0[x0_idx] = x0[x0_idx] + outcome
                            cur_cost = cost(x0, xdata, ydata, lambda_string_new)

                        for x0_idx in range(len(x0)):
                            test_x0 = x0.copy()
                            if absolute:
                                test_x0[x0_idx] = x0[x0_idx]-step_perc
                            else:
                                test_x0[x0_idx] = x0[x0_idx]*(1-step_perc)
                            new_cost = cost(test_x0, xdata, ydata, lambda_string_new)
                            if absolute:
                                outcome = ts.ckks_vector(context, [step_perc*(new_cost.decrypt()[0]<cur_cost.decrypt()[0])])                              
                            else:
                                outcome = ts.ckks_vector(context, [x0[x0_idx].decrypt()[0]*step_perc*(new_cost.decrypt()[0]<cur_cost.decrypt()[0])])
                            x0[x0_idx] = x0[x0_idx] - outcome
                            cur_cost = cost(x0, xdata, ydata, lambda_string_new)
                            
                res_x_list.append([i.decrypt() for i in x0])                          
                optimized_cost_temp = cost(x0, xdata, ydata, lambda_string_new)
                optimized_cost+=optimized_cost_temp
                lambda_string_list.append(lambda_string_new)
            ypred = np.array(ypred)
            ytrue = np.array(ytrue)
            optimized_cost = optimized_cost*(1/len(y))
            master_list.append((test_eq_orig,test_eq,lambda_string_list,res_x_list,optimized_cost))
        except RuntimeError:
            print("No fit found")
print(master_list)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:56<00:00, 11.61s/it]


[('Add(Pow(age,R),Pow(R,R))', 'Add(Pow(age,x[0]),Pow(x[1],x[2]))', ['lambda x,xdata:Add(Pow((xdata[0]),x[0]),Pow(x[1],x[2]))'], [[[0.49828212964937485], [1.6000000056191943], [2.4037952693075675]]], <tenseal.tensors.ckksvector.CKKSVector object at 0x0000014157A26DF0>)]
